In [1]:
import random
from bolib.Bo import Bo
from bolib.Dimension import NumericDimension
from bolib.ComputeSpace import ComputeSpace
from bolib.Normalizer import OptimizeFor
import pandas as pd
from IPython.display import clear_output
import numpy as np

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blackbox_dims = 6

In [3]:

def getRating(v, target):    
    diff = abs(np.array(v) - np.array(target))/(100*len(v))    
    return [sum(diff.tolist())]

def getSample(TARGET = None):
    s = [random.randint(0, 100) for _ in range(blackbox_dims)]
    if TARGET is not None:
        r= getRating(s, TARGET)
        return [s, r]
    return s


TARGET = getSample()

s, r = getSample(TARGET)
pd.DataFrame([[s, r, TARGET]], columns=["guess", "rating", "target"])  

,guess,rating,target
0,"[20, 32, 95, 2, 78, 54]",[0.3883333333333333],"[6, 67, 13, 80, 57, 57]"


In [4]:
numerics = [ NumericDimension(min=0, max=100, name=f"p{i}") for i in range(blackbox_dims) ]
ranking = [ NumericDimension(min=0, max=10, name="Ranking", optimize_for=OptimizeFor.MIN) ]

compSpace = ComputeSpace(numerics, ranking)


bo = Bo(compSpace)


def infer(x, y):  
  compSpace.add_values(xs = [x], ys=[y])   
  inf = bo.infer()
  denorm = compSpace.denormalize(inf)
  return [d.value for d in denorm]

infer(s,r)




[98.62150549888611,
 1.4877875335514545,
 9.97752919793129,
 79.55995202064514,
 6.819385290145874,
 2.2475266829133034]

In [5]:
last_rating = 0
last_guess = None
best_guess = None
best_rating = None
print(f"We are looking for {TARGET}")

df_cols = ["guess", "rating", "target"]
df = pd.DataFrame([], columns=df_cols)  

for i in range(20):
  last_rating = r[0]
  last_guess = s
  
  
  guess = infer(s,r)  
  
  r = getRating(guess, TARGET)

  tdf = pd.DataFrame([[s, r, TARGET]], columns=df_cols)  
  
  df = pd.concat([df, tdf], axis=0).reset_index(drop=True)
  
  clear_output(wait=True)

  print(df)

  
  #if r[0] == 2 
  """
  if r[0] < 0.15:
    print(f"Found it in {i} steps: {guess}")
    break
  """
  if best_guess is None:
    best_rating = r[0]
    best_guess = guess

  if r[0] < best_rating:
    best_guess = guess
    best_rating = r[0]
  s = guess

print(f"I think it is {best_guess}\n with a rating of {best_rating}")
print(TARGET)


  


  



                                                guess                 rating  \
0                             [20, 32, 95, 2, 78, 54]   [0.3175055990616481]   
1   [100.0, 100.0, 7.494154572486877, 85.067492723...  [0.42618901133537296]   
2   [100.0, 100.0, 0.0, 100.0, 0.0, 95.71340680122...  [0.27575299580891927]   
3   [100.0, 100.0, 11.937671899795532, 76.19284987...  [0.32122557676761065]   
4   [100.0, 100.0, 0.007370251842075959, 80.166482...  [0.30229756772518157]   
5   [100.0, 100.0, 26.286378502845764, 65.02233147...  [0.31994588454564415]   
6   [100.0, 100.0, 21.85235768556595, 75.482583045...  [0.27053198789556826]   
7   [100.0, 100.0, 4.408321529626846, 76.496517658...   [0.2518729684750239]   
8   [100.0, 100.0, 13.884228467941284, 84.93674397...  [0.23284728507200878]   
9   [100.0, 88.13791871070862, 13.074380159378052,...  [0.20982350001732508]   
10  [89.39019441604614, 83.00209641456604, 11.7500...  [0.20281779368718464]   
11  [85.57426929473877, 77.4830341339111

In [7]:
print(f"I think it is \n{best_guess} with a rating of {best_rating}")
print(TARGET)


I think it is 
[49.31081831455231, 68.94735097885132, 12.598812580108643, 79.98426556587219, 55.1800012588501, 55.4443895816803] with a rating of 0.08175116717815398
[6, 67, 13, 80, 57, 57]
